# MDD Symptoms meta analysis

Set up in process meta-analysis so that leave-one-out (LOO) can be swiftly generated.

In [2]:
import polars as pl

Symptom references

In [3]:
symptom_reference_tab = """
MDD1;depressed;Depressed mood most of the day, nearly every day
MDD2;anhedonia;Markedly diminished interest or pleasure in all, or almost all, activities most of the day, nearly every day
MDD3;weightChange;Significant change in weight or appetite
MDD3a;weightLoss;Significant weight loss or decrease in appetite
MDD3b;weightGain;Significant weight gain or increase in appetite
MDD4;sleepChange;Sleeping too much or not sleeping enough
MDD4a;sleepProb;Insomnia nearly every day
MDD4b;sleepMore;Hypersomnia nearly every day
MDD5;psychomotor;Changes in speed/amount of moving or speaking
MDD5a;psychomotorFast;Psychomotor agitation nearly every day
MDD5b;psychomotorSlow;Psychomotor retardation nearly every day
MDD6;fatigue;Fatigue or loss of energy nearly every day
MDD7;worthless;Feelings of worthlessness or excessive or inappropriate guilt
MDD8;concentration;Diminished ability to think or concentrate, or indecisiveness
MDD9;death;Recurrent thoughts of death or suicide or a suicide attempt or a specific plan for attempting suicide
"""

symptom_reference_dat = [line.split(';') for line in symptom_reference_tab.strip().split('\n')]
references = [symp[0] for symp in symptom_reference_dat]
abbrevs = [symp[1] for symp in symptom_reference_dat]
descriptions = [symp[0] for symp in symptom_reference_dat]
#dsm_mdd_symptoms_reference_df = pd.DataFrame(symptom_reference_dat,
#    columns=['Reference', 'Symptom Abbrev.', 'Description'])
dsm_mdd_symptoms_reference_df = pl.DataFrame(
    {
        "reference": references,
        "abbrev": abbrevs,
        "description": descriptions
    }
)
dsm_mdd_symptoms_reference_df

reference,abbrev,description
str,str,str
"""MDD1""","""depressed""","""MDD1"""
"""MDD2""","""anhedonia""","""MDD2"""
"""MDD3""","""weightChange""","""MDD3"""
"""MDD3a""","""weightLoss""","""MDD3a"""
"""MDD3b""","""weightGain""","""MDD3b"""
"""MDD4""","""sleepChange""","""MDD4"""
"""MDD4a""","""sleepProb""","""MDD4a"""
"""MDD4b""","""sleepMore""","""MDD4b"""
"""MDD5""","""psychomotor""","""MDD5"""


Read list of cohorts and groupings

In [4]:
cohorts_df = pl.read_csv("meta/cohort_alignment.txt", separator="\t")
cohorts_reference_df = cohorts_df.join(dsm_mdd_symptoms_reference_df, on="reference")

cohorts_reference_df.with_columns(
    pl.when(pl.col("cohort").is_in(["AGDS", "PGC", "GenScot", "janpy"]))
      .then(pl.lit("Clin"))
      .when(pl.col("cohort").is_in(["ASLPAC", "EstBB", "UKBB"]))
      .then(pl.lit("Comm"))
      .otherwise(pl.lit(None))
      .alias("metagroup"),
    pl.col("filename")
     .str.extract(r"daner_MDD[0-9ab]+_([a-z0-9]+)", group_index=1)
     .alias("studymatch")
).with_columns(
    (pl.col("metagroup") + "." + pl.col("reference") + "_" + pl.col("abbrev"))
     .alias("meta"),
    pl.when(pl.col("studymatch").is_null())
     .then(pl.col("cohort"))
     .otherwise(pl.col("studymatch"))
     .alias("studyname")
)

filename,cohort,reference,abbrev,description,metagroup,studymatch,meta,studyname
str,str,str,str,str,str,str,str,str
"""daner_MDD1_bid…","""PGC""","""MDD1""","""depressed""","""MDD1""","""Clin""","""bidi1""","""Clin.MDD1_depr…","""bidi1"""
"""daner_MDD1_gep…","""PGC""","""MDD1""","""depressed""","""MDD1""","""Clin""","""gep3""","""Clin.MDD1_depr…","""gep3"""
"""daner_MDD1_nes…","""PGC""","""MDD1""","""depressed""","""MDD1""","""Clin""","""nes1""","""Clin.MDD1_depr…","""nes1"""
"""daner_MDD1_rot…","""PGC""","""MDD1""","""depressed""","""MDD1""","""Clin""","""rot4""","""Clin.MDD1_depr…","""rot4"""
"""daner_MDD1_stm…","""PGC""","""MDD1""","""depressed""","""MDD1""","""Clin""","""stm2""","""Clin.MDD1_depr…","""stm2"""
"""daner_MDD1_twg…","""PGC""","""MDD1""","""depressed""","""MDD1""","""Clin""","""twg2""","""Clin.MDD1_depr…","""twg2"""
"""daner_MDD2_bid…","""PGC""","""MDD2""","""anhedonia""","""MDD2""","""Clin""","""bidi1""","""Clin.MDD2_anhe…","""bidi1"""
"""daner_MDD2_gen…","""PGC""","""MDD2""","""anhedonia""","""MDD2""","""Clin""","""gens""","""Clin.MDD2_anhe…","""gens"""
"""daner_MDD2_gep…","""PGC""","""MDD2""","""anhedonia""","""MDD2""","""Clin""","""gep3""","""Clin.MDD2_anhe…","""gep3"""
